In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [94]:
deliveries = pd.read_csv("deliveries.csv")
matches = pd.read_csv("matches.csv")

In [95]:
df = deliveries.groupby(by=["match_id","batting_team","bowling_team"])["total_runs"].sum()
df = pd.DataFrame(df).reset_index().set_index("match_id")


scores = pd.DataFrame(columns=["match_id","Teams","Toss Winner","Toss Decision","Team 1","Team 2","Team 1 Score","Team 2 Score","Team 1 Wickets","Team 2 Wickets"])
team_scores = {}
ctr = 0
for i in df.index.unique():
    #print(df.loc[i,"batting_team"])
    try:
        teams = df.loc[i,"batting_team"].values
        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        toss_win = teams[position_of_toss_winner]

        if toss_winner == teams[0]:
            if choice == "bat":
                first_batting_team = teams[0]
            elif choice == "field":
                first_batting_team = teams[1]
        else:
            if choice == "bat":
                first_batting_team = teams[1]
            elif choice == "field":
                first_batting_team = teams[0]
        second_batting_team = teams[1-np.where(teams==first_batting_team)[0][0]]
        p = teams[0] + " vs " + teams[1]
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = p
        scores.loc[ctr,"Team 1"] = first_batting_team
        scores.loc[ctr,"Team 2"] = second_batting_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match[particular_match["batting_team"]==first_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Team 2 Score"] = particular_match[particular_match["batting_team"]==second_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        #print(p)
        ctr += 1
        #break
    #print()
    except:
        batting_team = df.loc[i,"batting_team"]
        bowling_team = df.loc[i,"bowling_team"]

        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        #position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        #toss_win = teams[position_of_toss_winner]
        
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = batting_team + " vs " + bowling_team
        scores.loc[ctr,"Team 1"] = batting_team
        scores.loc[ctr,"Team 2"] = bowling_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match["total_runs"]
        scores.loc[ctr,"Team 2 Score"] = np.nan
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        ctr += 1

In [96]:
#deliveries.groupby(by=["match_id","batting_team","bowling_team"])["total_runs"].sum()
w = pd.DataFrame(columns=["Match ID","Batting Team","Bowling Team","Number of Wickets"])
ctr = 0

for i in deliveries.index.values[:-1]:
    current_batsman = deliveries.loc[i,"batsman"]
    next_batsman = deliveries.loc[i+1,"batsman"]

    current_striker = deliveries.loc[i,"non_striker"]
    next_striker = deliveries.loc[i+1,"non_striker"]

    if not ((current_batsman == next_batsman and current_striker == next_striker) or (current_batsman == next_striker and current_striker == next_batsman)):
        w.loc[ctr,"Match ID"] = deliveries.loc[i,"match_id"]
        w.loc[ctr,"Batting Team"] = deliveries.loc[i,"batting_team"]
        w.loc[ctr,"Bowling Team"] = deliveries.loc[i,"bowling_team"]
        w.loc[ctr,"Number of Wickets"] = 1
        ctr += 1

#w[(w["Match ID"] == 1) & (w["Batting Team"] == "Sunrisers Hyderabad") & (w["Bowling Team"] == "Royal Challengers Bangalore")]
wick = w.groupby(by=["Match ID","Batting Team","Bowling Team"])["Number of Wickets"].sum()
wick = wick.reset_index().set_index("Match ID")

In [97]:
wickets = pd.DataFrame(columns=["First Batting Team Wickets","Second Batting Team Wickets"])
ctr = 0
for i in wick.index.unique():
    try:
        teams = df.loc[i,"batting_team"].values
        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        toss_win = teams[position_of_toss_winner]

        if toss_winner == teams[0]:
            if choice == "bat":
                first_batting_team = teams[0]
            elif choice == "field":
                first_batting_team = teams[1]
        else:
            if choice == "bat":
                first_batting_team = teams[1]
            elif choice == "field":
                first_batting_team = teams[0]
        second_batting_team = teams[1-np.where(teams==first_batting_team)[0][0]]


        
        teams_1 = wick.loc[i,"Batting Team"].values
        number_of_wickets = wick.loc[i,"Number of Wickets"].values
        position_of_first_batting_team = np.where(teams_1 == first_batting_team)[0][0]
        

        number_of_wickets_first_batting_team = number_of_wickets[position_of_first_batting_team]
        position_of_second_batting_team = 1 - position_of_first_batting_team
        number_of_wickets_second_batting_team = number_of_wickets[position_of_second_batting_team]

        wickets.loc[ctr,"First Batting Team Wickets"] = number_of_wickets_first_batting_team
        wickets.loc[ctr,"Second Batting Team Wickets"] = number_of_wickets_second_batting_team
        ctr += 1
    except:
        teams_1 = wick.loc[i,"Batting Team"]
        number_of_wickets = wick.loc[i,"Number of Wickets"]

        wickets.loc[ctr,"First Batting Team Wickets"] = number_of_wickets
        ctr += 1

scores["Team 1 Wickets"] = wickets["First Batting Team Wickets"]
scores["Team 2 Wickets"] = wickets["Second Batting Team Wickets"]

In [98]:
scores.head()

,match_id,Teams,Toss Winner,Toss Decision,Team 1,Team 2,Team 1 Score,Team 2 Score,Team 1 Wickets,Team 2 Wickets
0,1,Royal Challengers Bangalore vs Sunrisers Hyder...,Royal Challengers Bangalore,field,Sunrisers Hyderabad,Royal Challengers Bangalore,207,172,5,10
1,2,Mumbai Indians vs Rising Pune Supergiant,Rising Pune Supergiant,field,Mumbai Indians,Rising Pune Supergiant,184,187,9,4
2,3,Gujarat Lions vs Kolkata Knight Riders,Kolkata Knight Riders,field,Gujarat Lions,Kolkata Knight Riders,183,184,5,1
3,4,Kings XI Punjab vs Rising Pune Supergiant,Kings XI Punjab,field,Rising Pune Supergiant,Kings XI Punjab,163,164,7,5
4,5,Delhi Daredevils vs Royal Challengers Bangalore,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,Delhi Daredevils,157,142,8,10
